In [1]:
import re
import pickle
import pandas as pd
import numpy as np
import scipy.sparse as sp
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from modules import tokenize_and_lemmatize

import warnings
warnings.simplefilter('ignore', FutureWarning)

[nltk_data] Downloading package wordnet to /Users/user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Preprocessing resumes

In [2]:
total_df = pd.read_csv('/Users/user/Documents/Magistracy/year_project/hr-ai-scout/total_df.csv')

resume_columns = [col for col in total_df.columns if 'resume_' in col]
df_resumes = total_df[resume_columns].drop_duplicates(ignore_index=True)

In [3]:
df_resumes = df_resumes.dropna(subset= ["resume_education",
                        "resume_last_experience_description",
                        "resume_last_position",
                        "resume_last_company_experience_period",
                        "resume_total_experience",
                        "resume_experience_months",
                        "resume_location",
                        "resume_specialization",
                       ], how="all")


df_resumes = df_resumes.loc[~(df_resumes["resume_total_experience"].notna()
        & df_resumes["resume_last_experience_description"].isna()
        & df_resumes["resume_last_position"].isna())]

In [4]:
num_cols = df_resumes.select_dtypes(include=[np.number]).columns
cat_cols = df_resumes.select_dtypes(include=['object']).columns

df_resumes[cat_cols] = df_resumes[cat_cols].fillna('NDT')

df_resumes['resume_age'] = df_resumes['resume_age'].fillna(df_resumes['resume_age'].mean())
df_resumes['resume_experience_months'] = df_resumes['resume_experience_months'].fillna(0)

In [5]:
df_resumes['resume_salary_split'] = df_resumes['resume_salary'].apply(lambda x: x.split())

df_resumes['salary_int'] = df_resumes['resume_salary_split'].apply(
    lambda x: int(''.join(part for part in x if re.fullmatch(r'\d+', part)))
              if any(re.fullmatch(r'\d+', part) for part in x)
              else np.nan
)

currency_symbols = ['₽', '$', '€', '₴', '₸', '₼', '₾', 'Br', "so'm"]

rates_rub = {
    "₽": 1.0,
    "$": 80.85,
    "€": 94.14,
    "₴": 1.94,
    "₸": 0.150,
    "₼": 47.8,
    "₾": 33.5,
    "Br": 28.7,
    "so'm": 0.0068
}

df_resumes['currency_symbol'] = df_resumes['resume_salary_split'].apply(
    lambda x: next((sym for sym in x if sym in currency_symbols), np.nan)
)

df_resumes['salary_converted'] = (df_resumes['salary_int'] * df_resumes['currency_symbol'].map(rates_rub)).fillna(0)

df_resumes['resume_salary'] = df_resumes['salary_converted']

df_resumes = df_resumes.drop(['resume_salary_split', 'salary_int', 'currency_symbol', 'salary_converted'], axis=1)

In [6]:
def experience_to_months(experience_text):
    months = 0
    # Опыт в годах
    years_match = re.search(r'(\d+)\s*год', experience_text)
    if years_match:
        months += int(years_match.group(1)) * 12

    years_match = re.search(r'(\d+)\s*лет', experience_text)
    if years_match:
        months += int(years_match.group(1)) * 12

    # Опыт в месяцах
    months_match = re.search(r'(\d+)\s*месяц', experience_text)
    if months_match:
        months += int(months_match.group(1))

    return months if months > 0 else 0

df_resumes['resume_last_company_experience_months'] = df_resumes['resume_last_company_experience_period'].apply(experience_to_months)

In [7]:
gender_map = {
    'Мужчина': 'Мужчина',
    'Male': 'Мужчина',
    'Женщина': 'Женщина',
    'Female': 'Женщина'
}

df_resumes['resume_gender'] = df_resumes['resume_gender'].apply(lambda x: gender_map[x] if x in gender_map else 'Неизвестно')

In [8]:
df_resumes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20718 entries, 0 to 20844
Data columns (total 17 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   resume_id                              20718 non-null  int64  
 1   resume_title                           20718 non-null  object 
 2   resume_specialization                  20718 non-null  object 
 3   resume_last_position                   20718 non-null  object 
 4   resume_last_experience_description     20718 non-null  object 
 5   resume_last_company_experience_period  20718 non-null  object 
 6   resume_skills                          20718 non-null  object 
 7   resume_education                       20718 non-null  object 
 8   resume_courses                         20718 non-null  object 
 9   resume_salary                          20718 non-null  float64
 10  resume_age                             20718 non-null  float64
 11  resume_

In [9]:
df_resumes = df_resumes.reset_index(drop=True)

In [10]:
df_resumes.to_csv('df_resumes.csv', index=False)

In [11]:
def get_tfidf_embeddings(texts, vectorizer=None, fit=True):
    """Создание TF-IDF эмбеддингов для списка текстов с лемматизацией"""
    if fit:
        vectorizer = TfidfVectorizer(
            max_features=5000,
            min_df=2,
            max_df=0.8,
            ngram_range=(1, 2),
            tokenizer=tokenize_and_lemmatize,
            token_pattern=None,
            lowercase=False  # Уже сделано в токенизации
        )
        embeddings = vectorizer.fit_transform(texts)
    else:
        embeddings = vectorizer.transform(texts)
    
    return embeddings, vectorizer

In [12]:
experience_embeddings, tfidf_vectorizer = get_tfidf_embeddings(df_resumes['resume_last_experience_description'].tolist(), vectorizer=None, fit=True)

In [13]:
MODEL_NAME = 'tfidf_vectorizer.pkl'
with open(MODEL_NAME, 'wb') as file:
    pickle.dump(tfidf_vectorizer, file)

In [14]:
sp.save_npz('experience_embeddings.npz', experience_embeddings, compressed=True)

In [17]:
total_df[total_df['vacancy_name'].str.contains('data')]

,vacancy_id,vacancy_name,vacancy_area,vacancy_experience,vacancy_employment,vacancy_schedule,vacancy_salary_from,vacancy_salary_to,vacancy_salary_currency,vacancy_salary_gross,...,resume_education,resume_courses,resume_salary,resume_age,resume_total_experience,resume_experience_months,resume_location,resume_gender,resume_applicant_status,target
7050,126415037,Аналитик данных/data analyst,Москва,От 1 года до 3 лет,Полная занятость,Полный день,80000.0,NaN,RUR,False,...,"['Skypro', 'Московский Технический Университет...",['Университет Иннополис'],NaN,NaN,6 лет 3 месяца,75.0,Москва,Мужчина,Активно ищет работу,1
7051,126415037,Аналитик данных/data analyst,Москва,От 1 года до 3 лет,Полная занятость,Полный день,80000.0,NaN,RUR,False,...,['Российский государственный аграрный универси...,['Skypro'],NaN,27.0,9 лет,108.0,Москва,Женщина,Активно ищет работу,1
7052,126415037,Аналитик данных/data analyst,Москва,От 1 года до 3 лет,Полная занятость,Полный день,80000.0,NaN,RUR,False,...,"['Geek University ', 'Московская Академия Экон...",NaN,250 000 ₽ на руки,41.0,16 лет 5 месяцев,197.0,Москва,Мужчина,Рассматривает предложения,1
7053,126415037,Аналитик данных/data analyst,Москва,От 1 года до 3 лет,Полная занятость,Полный день,80000.0,NaN,RUR,False,...,['Московский государственный университет путей...,"['Онлайн-обучение, Программист Java/Andriod/Ве...",NaN,47.0,27 лет 11 месяцев,335.0,Люберцы (Московская область),Мужчина,Рассматривает предложения,1
7054,126415037,Аналитик данных/data analyst,Москва,От 1 года до 3 лет,Полная занятость,Полный день,80000.0,NaN,RUR,False,...,"['Skyeng-Skypro', 'Современная гуманитарная ак...","['TTM Academy', 'ООО "" Хистори оф Пипл"" и ООО ...",85 000 ₽ in hand,49.0,30 лет,360.0,Moscow,Male,Рассматривает предложения,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323325,125492645,Middle+ data scientist,Москва,От 3 до 6 лет,Полная занятость,Полный день,NaN,NaN,NaN,NaN,...,['Московский государственный вечерний металлур...,"['Comozzi, Специалист по пнемолиниям']",350 000 ₽ на руки,38.0,19 лет 10 месяцев,238.0,Москва,Мужчина,Рассматривает предложения,0
323326,125492645,Middle+ data scientist,Москва,От 3 до 6 лет,Полная занятость,Полный день,NaN,NaN,NaN,NaN,...,"['Забайкальский государственный университет, Ч...",NaN,50 000 ₽ на руки,38.0,16 лет 6 месяцев,198.0,Москва,Мужчина,NaN,0
323327,125492645,Middle+ data scientist,Москва,От 3 до 6 лет,Полная занятость,Полный день,NaN,NaN,NaN,NaN,...,['Московский физико-технический институт (Госу...,NaN,90 000 ₽ на руки,32.0,1 год 8 месяцев,20.0,Москва,Мужчина,NaN,0
323328,125492645,Middle+ data scientist,Москва,От 3 до 6 лет,Полная занятость,Полный день,NaN,NaN,NaN,NaN,...,['Московский государственный технический униве...,NaN,60 000 ₽ на руки,61.0,30 лет 10 месяцев,370.0,Москва,Мужчина,NaN,0


In [18]:
total_df.loc[total_df['vacancy_id'] == 126415037, ['vacancy_description', 'resume_id', 'resume_title', 'target']]

,vacancy_description,resume_id,resume_title,target
7050,Твои задачи:• Формирование баз данных для КЦ (...,20903580,Аналитик данных (Data Analyst),1
7051,Твои задачи:• Формирование баз данных для КЦ (...,135124273,"Аналитик данных (SQL, Excel, Python)",1
7052,Твои задачи:• Формирование баз данных для КЦ (...,16673206,Data analyst/Python/SQL,1
7053,Твои задачи:• Формирование баз данных для КЦ (...,62159977,Бизнес-аналитик,1
7054,Твои задачи:• Формирование баз данных для КЦ (...,70281092,Аналитик данных,1
...,...,...,...,...
109305,Твои задачи:• Формирование баз данных для КЦ (...,46567798,"Главный инженер, заместитель главного инженера...",0
109306,Твои задачи:• Формирование баз данных для КЦ (...,41313063,Специалист,0
109307,Твои задачи:• Формирование баз данных для КЦ (...,57727098,Руководитель проектов,0
109308,Твои задачи:• Формирование баз данных для КЦ (...,660275,HSE,0


In [19]:
df_resumes[df_resumes['resume_id'] == 116651504]

,resume_id,resume_title,resume_specialization,resume_last_position,resume_last_experience_description,resume_last_company_experience_period,resume_skills,resume_education,resume_courses,resume_salary,resume_age,resume_total_experience,resume_experience_months,resume_location,resume_gender,resume_applicant_status,resume_last_company_experience_months


In [22]:
total_df.loc[total_df['vacancy_id'] == 126415037, 'vacancy_description']

array(['Твои задачи:• Формирование баз данных для КЦ (сбор, консолидация, проверка качества и актуальности).• Работа с источниками данных (поиск, анализ конверсии, отчётность) через собственную платформу.• Аналитика и отчётность (отчёты, выводы, рекомендации).• Поддержка контента и скриптов (анализ и корректировки).Что мы ждем:Hard skills:• Понимание метрик аналитики и маркетинга (CTR, CR, CPL, ROI, UTM).• • Навыки сбора, очистки и визуализации данных.. Продвинутое владение Excel / Google Таблицами (формулы, сводные таблицы, Power Query).· Опыт работы с системами визуализации данных (например, Power BI или Google Looker Studio) будет преимуществом.· Понимание ключевых маркетинговых метрик: конверсия (CR), стоимость лида (CPL), возврат на инвестиции (ROI).· Навыки работы с большими массивами информации: сбор, очистка, структурирование.Личностные Soft Skills:· Внимательность к деталям и аккуратность.· Системный и аналитический подход к решению задач.· Ответственность и нацеленность на ко